In [ ]:
import sys # for automation and parallelization: set manual to false when run by a launcher
manual, scenario = (True, 'base') if 'ipykernel' in sys.argv[0] else (False, sys.argv[1])

In [ ]:
import sys
sys.path.insert(0, r'../../../quetzal')

from quetzal.model import stepmodel
from quetzal.io import excel
import numpy as np
import pandas as pd

In [ ]:
sm = stepmodel.read_zippedpickles(r'../../model/{scen}/modal_split'.format(scen=scenario))

In [ ]:
from syspy.assignment import raw as raw_assignment
sm.compute_los_volume()

# PT ASSIGNMENT
# create PT volume column in sm.volumes
right = sm.probabilities[['origin', 'destination', 'pt']].rename(columns={'pt': 'pt_share'})
sm.volumes.drop(['pt_share', 'pt'], inplace=True, errors='ignore', axis=1)
sm.volumes = pd.merge(sm.volumes, right, on=['origin', 'destination'])
sm.volumes['pt'] = sm.volumes['pt_share'] * sm.volumes['todos']

# PT strategy assignment
sm.step_strategy_assignment(volume_column='pt', road=True)

# WALK ASSIGNMENT
walk_los = sm.los.loc[sm.los['route_type'] == 'walk']
sm.road_links['walk'] = raw_assignment.assign(
    list(walk_los['volume']), 
    list(walk_los['link_path'])
)

# CAR ASSIGNMENT
walk_los = sm.los.loc[sm.los['route_type'] == 'car']
sm.road_links['car'] = raw_assignment.assign(
    list(walk_los['volume']), 
    list(walk_los['link_path'])
)

In [ ]:
sm.to_zippedpickles(
    '../../model/{scen}/assignment'.format(scen=scenario), 
)

In [ ]:
end_of_notebook